# Use Custom Image, Software Specification and Runtime Definition to deploy a python function with `ibm-watson-machine-learning`

This notebook demonstrates how to deploy in Watson Machine Learning service a python function which requires to create custom software specification and runtime definition.  
Familiarity with oc and python is helpful. This notebook uses Python 3.10.


## Learning goals

The learning goals of this notebook are:

-  Creating a Custom Image
-  Creating custom software specification and runtime definition
-  Online deployment of python function
-  Scoring data using deployed function

## Contents

This notebook contains the following parts:

1.	[Setup](#setup)
2.  [Create custom image](#image)
4.	[Create software specification](#soft)
3.	[Create runtime definition](#runtime) 
4.	[Web service creation and scoring](#deploy)
5.  [Clean up](#cleanup)
6.	[Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Contact with your Cloud Pack for Data administrator and ask him for your account credentials

### Connection to WML

Authenticate the Watson Machine Learning service on IBM Cloud Pack for Data. You need to provide platform `url`, your `username` and `api_key`.

In [2]:
username = 'PASTE YOUR USERNAME HERE'
api_key = 'PASTE YOUR API_KEY HERE'
password = 'PASTE YOUR PASSWORD HERE'
url = 'PASTE THE PLATFORM URL HERE'
bedrock_url = 'PASTE THE PLATFORM BEDROCK URL HERE'

In [3]:
wml_credentials = {
    "username": username,
    "password": password,
    "url": url,
    "instance_id": 'openshift',
    "version": '4.8'
}

In [4]:
CPDHOST = url

### Install and import the `ibm-watson-machine-learning` package
**Note:** `ibm-watson-machine-learning` documentation can be found <a href="http://ibm-wml-api-pyclient.mybluemix.net/" target="_blank" rel="noopener no referrer">here</a>.

In [ ]:
!pip install -U ibm-watson-machine-learning

In [10]:
from ibm_watson_machine_learning import APIClient

client = APIClient(wml_credentials)

### Working with spaces

First of all, you need to create a space that will be used for your work. If you do not have space already created, you can use `{PLATFORM_URL}/ml-runtime/spaces?context=icp4data` to create one.

- Click New Deployment Space
- Create an empty space
- Go to space `Settings` tab
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cpd4.0/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below

In [ ]:
space_id = 'PASTE YOUR SPACE ID HERE'

You can use `list` method to print all existing spaces.

In [ ]:
client.spaces.list(limit=10)

To be able to interact with all resources available in Watson Machine Learning, you need to set **space** which you will be using.

In [12]:
client.set.default_space(space_id)

'SUCCESS'

<a id="image"></a>
## 2. Create a Custom Image

After the creation of a Custom image as described in https://www.ibm.com/docs/en/cloud-paks/cp-data/4.8?topic=deployments-deploying-using-custom-image. We need to create a new base software specification for the custom image and the runtime definition for it. This notebook shows how to perform these steps and use the custom image in a WML deployment:
1. Create a new base software specification for the custom image
2. Create the new runtime definition file for the custom image
3. Create and Deploy a WML `python function` model with the custom image



_Required Role: You must be a Cloud Pak for Data cluster administrator_ 


The Creation of the custom image involves the following steps, which cannot be done from a notebook. It involves pulling building pushing docker images and can be performed in the cluster.
1. Downloading the base image https://www.ibm.com/docs/en/cloud-paks/cp-data/4.8?topic=image-downloading-base
2. Creating and uploading a custom image https://www.ibm.com/docs/en/cloud-paks/cp-data/4.8?topic=image-creating-custom



#### 1. Downloading the base image


Identify the https://www.ibm.com/docs/en/cloud-paks/cp-data/4.8.x?topic=image-downloading-runtime-definition the runtime definition you want to use as base. e.g. `runtime-23.1-py3.10`


### Credentials and generating tokens

In [32]:
import requests, json
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [14]:
headers={'Authorization': f'Bearer {client._get_icptoken()}',
         'Content-Type': 'application/json'}
VERSION = '2022-02-01'

In [21]:
runtime_definition_name = "runtime-23.1-py3.10"

Write the chosen runtime definition to the file `<runtime-definition-name>-server.json`; `runtime-23.1-py3.10-server.json` in this example

In [23]:
import json

response = requests.get(
    f"{url}/v2/runtime_definitions?include=launch_configuration",
    headers=headers,
    verify=False
)

for r in response.json()["resources"]:
    if r["entity"]["name"] != runtime_definition_name:
        continue
    with open(f"{runtime_definition_name}-server.json", "w") as f:
        f.write(json.dumps(r["entity"], indent=4))
        template_runtime_definition = r["entity"]
        print(f"successfully saved to file: {runtime_definition_name}-server.json")

successfully saved to file: runtime-23.1-py3.10-server.json


We will now use the downloaded `<runtime-definition-name>-server.json` as the template

The required base image will be in the key `.launch_configuration.image`

In [31]:
print(f"required base image:\n\t{template_runtime_definition['launch_configuration']['image']}")

required base image:
	cp.icr.io/cp/cpd/wml-dep-rt23-1-ms@sha256:5005d7d2a7961533ffc9bf75dd5d7ea056ac7bd984022ca727ec0d1fd811d471


If you have access to the IBM Entitled registry URL, you can download the base image directly from the above, else you need to download from private container registry used at the time of installation.

Pull the base image: Here `cp.stg.icr.io/cp/cpd/` is my private container registry used at the time of installation
```
    podman login -u <user> -p <password> cp.stg.icr.io/cp/cpd/
    podman pull cp.stg.icr.io/cp/cpd/wml-dep-rt23-1-ms@sha256:5005d7d2a7961533ffc9bf75dd5d7ea056ac7bd984022ca727ec0d1fd811d471
```


#### 2. Creating and uploading a custom image

##### 2.1 Building the custom image
Dockerfile sample for this example is as below. We are going to install the program `jq` as well as the python package `pendulum` here.

```
ARG  base_image_tag
FROM ${base_image_tag}

# For installing OS packages, use root:root
# e.g.
USER root:root
RUN yum install -y jq

# For installing to conda site-packages,
# use wmlfuser:condausers
# e.g.
USER wmlfuser:condausers
RUN umask 002 && \
   pip install pendulum

```

Build the custom image `wml-demo-image:test-1`:
```
    podman build -t wml-demo-image:test-1 \
        --build-arg base_image_tag=cp.stg.icr.io/cp/cpd/wml-dep-rt23-1-ms@sha256:5005d7d2a7961533ffc9bf75dd5d7ea056ac7bd984022ca727ec0d1fd811d471 \
        -f Dockerfile
```

Some sanity checks on image for this example.
```
    podman run -it wml-demo-image:test-1 bash
    $ jq --version
    $ pip list | grep pendulum
    $ uwsgi --version
    $ exit
```

##### 2.2 Uploading the custom image

Push the custom image to the private container registry
```
    e.g
    podman login -u <user> -p <password> cp.stg.icr.io/cp/cpd/
    podman tag wml-demo-image:test-1 cp.stg.icr.io/cp/cpd/wml-demo-image:test-1
    podman push cp.stg.icr.io/cp/cpd/wml-demo-image:test-1
    
    Required custom image:
    cp.stg.icr.io/cp/cpd/wml-demo-image:test-1
```



## Create a Volume in an existing PVC - "cc-home-pvc"

Creation of Base software specification and Runtime definition require uploading of the metadata json files to certain PVC path. We will use Volume Service APIs to access the PVC `cc-home-pvc`

`namespace` here is where cc-home-pvc lives
`oc get pvc -A | grep cc-home-pvc`

In [34]:
# name for this Vol Service instance
display_name='CCHome'
namespace="zen"

In [35]:
response = requests.post(
    CPDHOST + '/zen-data/v3/service_instances',
    headers=headers,
    json={
        "addon_type": "volumes",
        "addon_version": "-",
        "create_arguments": {
            "metadata": {
                "existing_pvc_name": "cc-home-pvc"
            }
        },
        "namespace": namespace,
        "display_name": display_name
    },
    verify=False)
response.status_code, response.json()

(409,
 {'errors': [{'code': 'service_instance_exist',
    'target': {'name': 'field', 'type': 'display_name'}}],
  'status_code': 409,
  'trace': 'f2162210-6909-471f-96ac-97f6a70ee91b'})

### Check Volume Instances - if it is already created

In [41]:
response = requests.get(CPDHOST + '/zen-data/v3/service_instances',
                        headers=headers,
                        verify=False)
response.status_code
vol_svc_id = None
for svc_inst in response.json()['service_instances']:
    print(svc_inst['display_name'], svc_inst['id'], svc_inst['metadata'].get('existing_pvc_name'))
    if svc_inst['display_name'] == f"{namespace}::{display_name}":
        vol_svc_id = svc_inst['id']

assert vol_svc_id is not None
print('\nvol_svc_id:', vol_svc_id)

zen::CCHome 1696949590847933 cc-home-pvc
zen::cc-homies 1696511688591187 cc-home-pvc
zen::wmlTrainingData 1696425781096032 file-api-claim
zen::dep-fvt 1696401827090487 None
zen::dep-fvt 1696401825654932 None

vol_svc_id: 1696949590847933


### Check the new volume instance

In [42]:
response = requests.get(CPDHOST + f'/zen-data/v3/service_instances/{vol_svc_id}',
                        headers=headers,
                        verify=False)
print(json.dumps(response.json(), indent=2))

{
  "service_instance": {
    "addon_type": "volumes",
    "addon_version": "-",
    "created_at": "2023-10-10T14:53:10.850651Z",
    "display_name": "zen::CCHome",
    "id": "1696949590847933",
    "instance_identifiers": null,
    "metadata": {
      "existing_pvc_name": "cc-home-pvc"
    },
    "misc_data": {},
    "namespace": "zen",
    "owner_uid": "1000330999",
    "owner_username": "admin",
    "parameters": {},
    "provision_status": "PROVISIONED",
    "resources": {
      "cpu": "0.30",
      "memory": "0.25 Gi"
    },
    "roles": [
      "Admin"
    ],
    "updated_at": "2023-10-10T14:53:10.868486Z",
    "zen_service_instance_info": {
      "docker_registry_prefix": "icr.io/cpopen/cpfs"
    }
  }
}


### Start file server on volume

In [46]:
response = requests.post(
    CPDHOST + f'/zen-data/v1/volumes/volume_services/{namespace}::{display_name}',
    headers=headers,
    json={},
    verify=False)
print(json.dumps(response.json(), indent=2))

{
  "_statusCode_": 409,
  "exception": "deployment cchome already exists",
  "message": "Conflict"
}


In [47]:
# In case you want to restart
response = requests.delete(
    CPDHOST + f'/zen-data/v1/volumes/volume_services/{namespace}::{display_name}',
    headers=headers,
    json={},
    verify=False)
# print(json.dumps(response.json(), indent=2))
response = requests.post(
    CPDHOST + f'/zen-data/v1/volumes/volume_services/{namespace}::{display_name}',
    headers=headers,
    json={},
    verify=False)
print(json.dumps(response.json(), indent=2))

{
  "_messageCode_": "200",
  "message": "Successfully started volume service"
}


<a id='soft'></a>
# 3. Create a new base software specification for the custom image

We will create the base software specification `cspec-rt23.1-test` for this demo

In [49]:
sw_spec_name='cspec-rt23.1-test'
sw_spec_filename=f'{sw_spec_name}.json'
print('software spec name     :', sw_spec_name)
print('software spec filename :', sw_spec_filename)

software spec name     : cspec-rt23.1-test
software spec filename : cspec-rt23.1-test.json


####  Custom Image software specification:
- `entity.software_specification.type` have to be set to "base" to indicate custom image.
- `entity.software_specification.built_in` have to be set boolean `False` to indicate custom image.
- `entity.software_configuration.included_packages` array is informational, it can be empty list `[]` . 
- `entity.software_configuration.platform.name` should be set to `python` .

In [50]:
sw_spec = {
  "metadata": {
    "name": sw_spec_name,
    "description": "Test custom image base software specification"
  },
  "entity": {
    "software_specification": {
      "type": "base",
      "built_in": False,
      "package_extensions": [],
      "display_name": f"Test {sw_spec_name}",
      "software_configuration": {
        "included_packages": [
          {
            "name": "pendulum",
            "version": "1.0.0"
          },
          {
            "name": "dummy-b",
            "version": "1.0.0"
          }         
        ],
        "platform": {
          "name": "python",
          "version": "3.10"
        }
      }
    }
  }
}

### Upload file to the specified path on the volume

In [51]:
target_file=f'%2F_global_%2Fconfig%2Fenvironments%2Fsoftware-specifications%2F{sw_spec_filename}'
target_file

'%2F_global_%2Fconfig%2Fenvironments%2Fsoftware-specifications%2Fcspec-rt23.1-test.json'

In [53]:
# multipart/form-data
# Do not set the Content-type header yourself, leave that to pyrequests to generate

response = requests.put(
    CPDHOST + f'/zen-volumes/{display_name}/v1/volumes/files/{target_file}',
    headers={
        "Authorization": f"Bearer {client._get_icptoken()}",
    },
    files={
        'upFile': (sw_spec_filename, json.dumps(sw_spec), 'application/json')
    },
    verify=False)
response.status_code, response.json()

(200,
 {'_messageCode_': 'Success',
  'message': 'Successfully uploaded file and created the directory structure'})

### We can check if the file was successfully upload

In [54]:
response = requests.get(CPDHOST + f'/zen-volumes/{display_name}/v1/volumes/files/{target_file}',
                        headers=headers,
                        verify=False)
print(response.status_code, '\n', json.dumps(response.json(), indent=2))

200 
 {
  "metadata": {
    "name": "cspec-rt23.1-test",
    "description": "Test custom image base software specification"
  },
  "entity": {
    "software_specification": {
      "type": "base",
      "built_in": false,
      "package_extensions": [],
      "display_name": "Test cspec-rt23.1-test",
      "software_configuration": {
        "included_packages": [
          {
            "name": "pendulum",
            "version": "1.0.0"
          },
          {
            "name": "dummy-b",
            "version": "1.0.0"
          }
        ],
        "platform": {
          "name": "python",
          "version": "3.10"
        }
      }
    }
  }
}


### Wait for about 5 mins
Refresh intervel is handled by `env-spec-sync-job` cronjob
```
e.g.
oc get cronjobs env-spec-sync-job

NAME                SCHEDULE      SUSPEND   ACTIVE   LAST SCHEDULE   AGE
env-spec-sync-job   */5 * * * *   False     0        82s             23d
```

### Check if the new Software Specification is activated

In [55]:
response = requests.get(
    CPDHOST + f'/v2/software_specifications?name={sw_spec_name}',
    headers=headers,
    verify=False)
print(json.dumps(response.json(), indent=2))

{
  "total_results": 0,
  "resources": []
}


### **Warning**: Do not proceed unless the new software specification is activated!

In [59]:
response = requests.get(
    CPDHOST + f'/v2/software_specifications?name={sw_spec_name}',
    headers=headers,
    verify=False)
print(json.dumps(response.json(), indent=2))

{
  "total_results": 1,
  "resources": [
    {
      "metadata": {
        "created_at": "2023-10-11T15:00:03.600Z",
        "updated_at": "2023-10-11T15:00:03.600Z",
        "name": "cspec-rt23.1-test",
        "description": "Test custom image base software specification",
        "asset_type": "software_specification",
        "href": "/v2/software_specifications/3e724cac-4db6-552a-9422-c304af09b1b0",
        "asset_id": "3e724cac-4db6-552a-9422-c304af09b1b0"
      },
      "entity": {
        "software_specification": {
          "type": "base",
          "built_in": false,
          "package_extensions": [],
          "display_name": "Test cspec-rt23.1-test",
          "software_configuration": {
            "included_packages": [
              {
                "name": "pendulum",
                "version": "1.0.0"
              },
              {
                "name": "dummy-b",
                "version": "1.0.0"
              }
            ],
            "platform": {
       

<a id="runtime"></a>
# 4. Create the new runtime definition file for the custom image

We will create the Runtime definition file, which connects the software specification `cspec-rt23.1-test` to the custom image name `cp.stg.icr.io/cp/cpd/wml-demo-image:test-1`

In [60]:
# fully qualified name for the Internal Registry image of custom image
custom_image_fqn='cp.stg.icr.io/cp/cpd/wml-demo-image:test-1'
custom_image_fqn

'cp.stg.icr.io/cp/cpd/wml-demo-image:test-1'

## Create a new runtime definition for the custom image

#### Runtime Definition file for Custom image:
- We no longer upload files for the volument service. We create runtime definitions via API `/v2/runtime_definitions`
- This runtime definition is the link between the software specification and the image name for custom image
- If the custom image had been uploaded to the private container registry, then the value for `custom_image_fqn` would be `cp.stg.icr.io/cp/cpd/wml-demo-image:test-1`

We need to use the `<runtime-definition>-server.json` as a template which we used earlier to get the base image from. Update the follow fields in the template

In [66]:
template_runtime_definition["name"] = sw_spec_name
template_runtime_definition["display_name"] = sw_spec_name
template_runtime_definition["description"] = f"Runtime definition for custom image {sw_spec_name}"
template_runtime_definition["built_in"] = True
template_runtime_definition["runtime_type"] = "wml"
template_runtime_definition["launch_configuration"]["software_specification_name"] = sw_spec_name
template_runtime_definition["launch_configuration"]["image"] = custom_image_fqn

custom_image_runtime_definition = template_runtime_definition

In [70]:
response = requests.post(
    CPDHOST + f'/v2/runtime_definitions', 
    headers=headers,
    json=custom_image_runtime_definition,
    verify=False)
response.status_code

201

In [71]:
response.json()

{'metadata': {'guid': '5dad9432-faa5-5d2e-b711-55cb9d260569',
  'created_at': '2023-10-11T15:20:45.278Z'},
 'entity': {'name': 'cspec-rt23.1-test',
  'display_name': 'cspec-rt23.1-test',
  'description': 'Runtime definition for custom image cspec-rt23.1-test',
  'author': 'IBM',
  'tested': True,
  'is_service': True,
  'built_in': False,
  'features': ['wml'],
  'runtime_type': 'wml',
  'software_configuration': {}}}

### Check if the new Runtime Definition is activated

In [74]:
response = requests.get(
    CPDHOST + f'/v2/runtime_definitions', 
    headers=headers,
    verify=False)
for r in response.json()['resources']:
    if r['entity']['runtime_type'] == 'wml' and r['entity']['name'] == run_def_name:
        print("Found:",json.dumps(r, indent=2))

Found: {
  "metadata": {
    "guid": "5dad9432-faa5-5d2e-b711-55cb9d260569",
    "created_at": "2023-10-11T15:20:45.278Z"
  },
  "entity": {
    "name": "cspec-rt23.1-test",
    "display_name": "cspec-rt23.1-test",
    "description": "Runtime definition for custom image cspec-rt23.1-test",
    "author": "IBM",
    "tested": true,
    "is_service": true,
    "built_in": false,
    "features": [
      "wml"
    ],
    "runtime_type": "wml",
    "software_configuration": {}
  }
}


In [75]:
print('The required software specification name:', sw_spec_name)
print('\t', custom_image_fqn)

The required software specification name: cspec-rt23.1-test
	 cp.stg.icr.io/cp/cpd/wml-demo-image:test-1


### Stop File server on the Volume

In [76]:
response = requests.delete(
    CPDHOST + f'/zen-data/v1/volumes/volume_services/{namespace}:{display_name}',
    headers=headers,
    json={},
    verify=False)
print(json.dumps(response.json(), indent=2))

{
  "_statusCode_": 404,
  "exception": "RECORD_NOT_FOUND",
  "message": "Not Found"
}


### Cleanup - delete the Volume instance

In [77]:
response = requests.delete(CPDHOST + f'/zen-data/v3/service_instances/{vol_svc_id}',
                           headers=headers,
                           verify=False)
print(json.dumps(response.json(), indent=2))

{
  "id": "1696949590847933"
}


<a id="deploy"></a>

## 5. Create and Deploy a WML `python function` model with the custom image

The demo `python function` below showcase the package `simplejson` which was installed for this custom image is now usable by returning the `version`, and does nothing else useful beyond that.

In [78]:
%%writefile aifun_{sw_spec_name}.py

def score_generator():
        
    # Define scoring function
    def callModel(payload_scoring):
        import pendulum

        v4_scoring_response = {
            'predictions': [{'values': [pendulum.__version__]}]
        }
    
        return v4_scoring_response

    def score(input):

        # Score using the pre-defined model
        prediction = callModel(input);
        return prediction
    
    return score


score = score_generator()

Writing aifun_cspec-rt23.1-test.py


`python function` needs to be gzipped for storing in WML repository

In [79]:
!gzip -fk aifun_{sw_spec_name}.py

In [80]:
!ls aifun_{sw_spec_name}*

aifun_cspec-rt23.1-test.py    aifun_cspec-rt23.1-test.py.gz


**We need use the Base software specification that was created earlier `cspec-38-1-t1`**

In [81]:
sw_spec_name, client.software_specifications.get_id_by_name(sw_spec_name)

('cspec-rt23.1-test', '3e724cac-4db6-552a-9422-c304af09b1b0')

In [82]:
meta_props = {
   client.repository.FunctionMetaNames.NAME: "py-function for custom image test",
   client.repository.FunctionMetaNames.DESCRIPTION: "Test Custom Image",
   client.repository.FunctionMetaNames.SOFTWARE_SPEC_UID:
        client.software_specifications.get_id_by_name(sw_spec_name)
}
stored_function_details = client.repository.store_function(f'aifun_{sw_spec_name}.py.gz', meta_props)
stored_function_details

{'entity': {'software_spec': {'id': '3e724cac-4db6-552a-9422-c304af09b1b0',
   'name': 'cspec-rt23.1-test'},
  'type': 'python'},
 'metadata': {'created_at': '2023-10-11T15:23:10.113Z',
  'description': 'Test Custom Image',
  'id': '14a53356-9a34-485a-9c85-1a9fbf464090',
  'modified_at': '2023-10-11T15:23:10.113Z',
  'name': 'py-function for custom image test',
  'owner': '1000330999',
  'space_id': '52f54f95-1887-421a-94dd-d2e77b98201c'},
 'system': {'warnings': []}}

In [83]:
function_id=client.repository.get_function_id(stored_function_details)
function_id

'14a53356-9a34-485a-9c85-1a9fbf464090'

### Create an online deployment

In [84]:
meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: "function dep Online custom image test",
    client.deployments.ConfigurationMetaNames.ONLINE: {},
    client.deployments.ConfigurationMetaNames.HARDWARE_SPEC : {
        "id":  client.hardware_specifications.get_id_by_name('S')}
}
deployment_details = client.deployments.create(function_id, meta_props)



#######################################################################################

Synchronous deployment creation for uid: '14a53356-9a34-485a-9c85-1a9fbf464090' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
.....
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='2e4afc5b-e321-45a4-95e8-1a0086a61dfc'
------------------------------------------------------------------------------------------------




You can monitor the cluster for pods with `oc get pods -l WML_DEPLOYMENT_ID=<deployment_id>`


example:
```
$ oc get pods -l WML_DEPLOYMENT_ID=2e4afc5b-e321-45a4-95e8-1a0086a61dfc
NAME                                                       READY   STATUS    RESTARTS   AGE
wml-2c074a33-8960-47a0-9489-1bd050bbe901-b9867df6f-9fr9g   2/2     Running   0          64s

$ oc describe pod wml-2c074a33-8960-47a0-9489-1bd050bbe901-b9867df6f-9fr9g | grep Image: | grep wml-demo-image
    Image:          cp.stg.icr.io/cp/cpd/wml-demo-image:test-1
```

In [85]:
deployment_id=client.deployments.get_uid(deployment_details)
deployment_id

'2e4afc5b-e321-45a4-95e8-1a0086a61dfc'

### Predict using created deployment

In [86]:
scoring_payload = {
    'input_data': [
        { 'id': 'id1', 'fields': ['dummy', 'dummy_c2'], 'values': [[1, 1]]}
    ]
}

In [87]:
predictions = client.deployments.score(deployment_id, scoring_payload)
predictions

{'predictions': [{'values': ['2.1.2']}]}

<a id="cleanup"></a>
## 6. Clean up   

In [88]:
client.deployments.delete(deployment_id)

'SUCCESS'

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watson-machine-learning-samples/blob/master/cpd4.0/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
## 7. Summary and next steps     

 You successfully completed this notebook! You learned how to use Watson Machine Learning for function deployment and scoring with custom custom_image, software_spec and runtime definition.  
 Check out our [Online Documentation](https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/welcome-main.html?context=analytics) for more samples, tutorials, documentation, how-tos, and blog posts. 

#### References:
- https://www.ibm.com/docs/en/cloud-paks/cp-data/4.7.x?topic=runtimes-customizing-deployment-runtime-images
- https://www.ibm.com/docs/en/cloud-paks/cp-data/4.7.x?topic=resources-volumes-api

### Author

**Ginbiaksang Naulak** Software Engineer at IBM.

Copyright © 2020, 2021, 2022, 2023 IBM. This notebook and its source code are released under the terms of the MIT License.